In [1]:
from src import run_experiments as re
from motif.contour_extractors.salamon import Salamon
from motif.run import process
import properties as prop
import motif.core as core
import numpy as np
import csv
import json
import os

outdir = os.getcwd() + prop.relOutputDir
MEL_TYPE = 1
NO_EXPERIMENTO = 5

In [2]:

"""
Todos os pares (arquivo_de_notacao nome_arquivo_audio_MELODY{1,2,3}.csv, nome_arquivo_audio.WAV) são 
selecionados de experiment_splits.json, da chave NO_EXPERIMENTO (1 a 5) e da subchave 'train', guardados
na variável pares
e os arquivos de teste provenientes da subchave 'test' são guardados em audios_file
e todos as musicas utilizadas nos testes e treino sao gravadas em track_list
"""
def get_audio_files_training_pairs_track_list(experiment_number = 1, mel_type = 1):
    #pares de treinamento vindos do MEDLEY
    annotationDir = prop.melodyRoot + str(mel_type) + "/"
    audio_root = prop.audioRoot
    SUFIX = "_MELODY" + str(mel_type) + ".csv"
    EXPERIMENT_NUMBER_STR = str(experiment_number)
    track_list = []
    training_pairs = []
    audio_files = [] 
    jsonDir = os.getcwd().split(prop.projectDirName)[0] + prop.projectDirName + "/" + prop.testDataName + "/"
    jsonFile = jsonDir + prop.experimentSplits
    with open(jsonFile, 'r') as fhandle:
        all_track_list = json.load(fhandle)
        for track in all_track_list[EXPERIMENT_NUMBER_STR]['train']:
            annotation = annotationDir + track + SUFIX
            audio_file = audio_root + track + "/" + track + "_MIX.wav"
            training_pairs.append((audio_file, annotation))
            track_list.append(track)
        for track in all_track_list[EXPERIMENT_NUMBER_STR]['test']:
            audio_files.append(audio_root + track + "/" + track + "_MIX.wav")
            track_list.append(track)
    #todos os caminhos de experiment_splits.json
#     with open(jsonFile, 'r') as fhandle:
#         all_track_list = json.load(fhandle)
#         for track in all_track_list[EXPERIMENT_NUMBER_STR]['test']:
#             audio_files.append(audio_root + track + "/" + track + "_MIX.wav")
            
    return audio_files, training_pairs, track_list

In [3]:
def extract_contours(audio_files, training_pairs, track_list):
    #extrai os contornos dos arquivos 
    contours_root = prop.contoursRoot
    
    for nome in(track_list):
        # cria o diretorio
        contour_dir = contours_root + nome
        try:
            os.stat(contour_dir)
        except:
            os.mkdir(contour_dir)

    results = process(
            audio_files=audio_files, training_pairs=training_pairs,
            testing_pairs=None, extract_id='salamon',
            feature_id='melodia', classifier_id='random_forest')
    
    #results = {training contours, test contours + test features extracted}
    return results


In [4]:
def salva(array, output_fpath):
    with open(output_fpath, 'w') as fhandle:
        writer = csv.writer(fhandle, delimiter=',')
        writer.writerows(item for item in array)
        
def save_contours_and_features(results):
    contours_root = prop.contoursRoot
    for treinada in results[0]:
        caminho = contours_root + treinada.audio_filepath.split('Audio')[1][0:-4] + "_TREINO.csv"
        treinada.save(caminho)
    """
     salvando os contornos obtidos e as features 
     (onset', 'offset', 'duration', 'pitch mean', 'pitch std',
     'salience mean', 'salience std', 'salience tot',
     'vibrato', 'vib rate', 'vib extent', 'vib coverage')
     e fazendo uma matrix que em cada linha coloca todas os tempo, freqs, saliencias
     dentro do intervalo de onset e offset
    """
    novaMatriz = []
    dimMax = 0
    for resultado in results[1]:
        caminho = contours_root + resultado[0].audio_filepath.split('Audio')[1][0:-4] + ".csv"
        #objeto contorno
        contorno = resultado[0]
        contorno.save(caminho)
        #features
        features = resultado[1]
        probs = np.asarray(resultado[2])
        probsT = np.resize(probs, (probs.shape[0],1))
        featuresProbs = np.concatenate((features, probsT), axis=1)
        caminho_csv = caminho[0:-4] + "_RESULT.csv"
        #fazendo o mix
        #cria uma lista de copia
        novaMatriz = featuresProbs.tolist()
        dimMax = 0
        for featureIndex in range(featuresProbs.shape[0]):
            linha = novaMatriz[featureIndex]
            onset = linha[0]
            offset = linha[1]
            for timeIndex in range(len(contorno.times)):
                time = contorno.times[timeIndex]
                if(onset <= time and time <= offset):
                    freq = contorno.freqs[timeIndex]
                    sals = contorno.salience[timeIndex]
                    linha.append(time)
                    linha.append(freq)
                    linha.append(sals)
                    del novaMatriz[featureIndex]
                    novaMatriz.insert(featureIndex, linha)
                    dimMax = len(linha) if len(linha) > dimMax else dimMax
#                 elif(time > offset):
#                     break
        #preenchendo as linhas com NaN para a matriz ficar completa 
        nan = float('nan')
        for linha in novaMatriz:
            tamanho = len(linha)
            while(tamanho < dimMax):
                linha.append(nan)
                tamanho += 1
        novaMatriz = np.asarray(novaMatriz)
        salva(novaMatriz, caminho_csv)


In [6]:
for no_exper in range(1,5):
    print("Experimento ", no_exper)
    audios, trainings, tracks = get_audio_files_training_pairs_track_list(no_exper, MEL_TYPE)
    print("Extraindo contornos...")
    results = extract_contours(audios, trainings, tracks)
    print("Salvando...")
    save_contours_and_features(results)

Experimento  1
Extraindo contornos...
Salvando...
Experimento  2
Extraindo contornos...

/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:231: UserWarning: Non-uniform timescale passed to resample_melody_series.  Pitch will be linearly interpolated, which will result in undesirable behavior if silences are indicated by missing values.  Silences should be indicated by nonpositive frequency values.
  "Non-uniform timescale passed to resample_melody_series.  Pitch "
/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:77: UserWarning: Reference melody has no voiced frames.
  warnings.warn("Reference melody has no voiced frames.")
/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:231: UserWarning: Non-uniform timescale passed to resample_melody_series.  Pitch will be linearly interpolated, which will result in undesirable behavior if silences are indicated by missing values.  Silences should be indicated by nonpositive 


Salvando...
Experimento 

/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:77: UserWarning: Reference melody has no voiced frames.
  warnings.warn("Reference melody has no voiced frames.")
/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:231: UserWarning: Non-uniform timescale passed to resample_melody_series.  Pitch will be linearly interpolated, which will result in undesirable behavior if silences are indicated by missing values.  Silences should be indicated by nonpositive frequency values.
  "Non-uniform timescale passed to resample_melody_series.  Pitch "


 3
Extraindo contornos...
Salvando...
Experimento 

/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:77: UserWarning: Reference melody has no voiced frames.
  warnings.warn("Reference melody has no voiced frames.")
/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:231: UserWarning: Non-uniform timescale passed to resample_melody_series.  Pitch will be linearly interpolated, which will result in undesirable behavior if silences are indicated by missing values.  Silences should be indicated by nonpositive frequency values.
  "Non-uniform timescale passed to resample_melody_series.  Pitch "


 4
Extraindo contornos...
Salvando...


/Volumes/Dados/Google Drive/Estudos/Mestrado/workspace/python/contour_classification/mir_eval/melody.py:77: UserWarning: Reference melody has no voiced frames.
  warnings.warn("Reference melody has no voiced frames.")
